# Understanding Stateful LSTM Recurrent Neural Networks

A powerful and popular recurrent neural network is the long short-term model network or LSTM. It is widely used because the architecture overcomes the vanishing and exploding gradient problem that plagues all recurrent neural networks, allowing very large and very deep networks to be created. Like other recurrent neural networks, LSTM networks maintain state, and the specifics of how this is implemented in the Keras framework can be confusing. This lesson will show exactly how the state is maintained in LSTM networks by the Keras deep learning library. After reading this lesson, you will know:
* How to develop a naive LSTM network for a sequence prediction problem.
* How to carefully manage state through batches and features with an LSTM network.
* How to manually manage state in an LSTM network for stateful prediction.

Let's get started.

## Problem Description: Learn the Alphabet

In this tutorial, we will develop and contrast a number of different LSTM recurrent neural network models. The context of these comparisons will be a simple sequence prediction problem of learning the alphabet. That is, given a letter of the alphabet, predict the next letter of the alphabet. Once understood, this is a simple sequence prediction problem that can be generalized to other sequence prediction problems like time series prediction and sequence classification. Let's prepare the problem with some Python code that we can reuse from example to example. Firstly, let's import all of the classes and functions we plan to use in this tutorial.


In [1]:
## Listing 27.1

Next, we can seed the random number generator to ensure that the results are the same each time the code is executed.

In [2]:
## Listing 27.2

We can now define our dataset, the alphabet. We define the alphabet in uppercase characters for readability. Neural networks model numbers, so we need to map the letters of the alphabet to integer values. We can do this easily by creating a dictionary (map) of the letter index to the character. We can also create a reverse lookup for converting predictions back into characters to be used later.

In [3]:
## Listing 27.3

Now we need to create our input and output pairs on which to train our neural network. We can do this by defining an input sequence length, then reading sequences from the input alphabet sequence. For example, we use an input length of 1. Starting at the beginning of the raw input data, we can read off the first letter A and the next letter as the prediction B. We move along one character and repeat until we reach a prediction of Z.

In [4]:
## Listing 27.4

We also print out the input pairs for sanity checking. Running the code will produce the following output, summarizing input sequences of length one and a single output character.

In [5]:
## Listing 27.5

We need to reshape the NumPy array into a format expected by the LSTM networks [samples, time steps, features].

In [6]:
## Listing 27.6

Once reshaped, we can then normalize the input integers to the range 0-to-1, the range of the sigmoid activation functions used by the LSTM network.

In [7]:
## Listing 27.7

Finally, we can think of this problem as a sequence classification task, where each of the 26 letters represents a different class. As such, we can convert the output (y) to one-hot encoding, using the Keras built-in function `to_categorical()`.

In [8]:
## Listing 27.8

We are now ready to fit different LSTM models.

## LSTM for Learning One-Char to One-Char Mapping

Let's start by designing a simple LSTM to learn how to predict the next character in the alphabet, given the context of just one character. We will frame the problem as a random collection of one-letter input to one-letter output pairs. As we will see, this is a difficult framing of the problem for the LSTM to learn. Let's define an LSTM network with 32 units and an output layer using the softmax activation function for making predictions. Because this is a multiclass classification problem, we can use the log loss function (called `categorical_crossentropy` in Keras), and optimize the network using the ADAM optimization function. The model is fit over 500 epochs with a batch size of 1.

In [9]:
## Listing 27.9

After we fit the model, we can evaluate and summarize the performance on the entire training dataset.

In [10]:
## Listing 27.10

We can then re-run the training data through the network and generate predictions, converting both the input and output pairs back into their original character format to visualize how well the network learned the problem.

In [11]:
## Listing 27.11

The entire code listing is provided below for completeness.

In [12]:
## Listing 27.12

Running this example produces the following output.

In [13]:
## Listing 27.13

We can see that this problem is indeed difficult for the network to learn. The reason is, the poor LSTM units do not have any context to work with. Each input-output pattern is shown to the network in a random order, and the state of the network is reset after each pattern (each batch where each batch contains one pattern). This is an abuse of the LSTM network architecture, treating it like a standard Multilayer Perceptron. Next, let's try a different framing of the problem to provide more sequence to the network from which to learn.

## LSTM for a Feature Window to One-Char Mapping

A popular approach to adding more context to data for Multilayer Perceptrons is to use the window method. This is where previous steps in the sequence are provided as additional input features to the network. We can try the same trick to provide more context to the LSTM network. Here, we increase the sequence length from 1 to 3, for example:

In [14]:
## Listing 27.14

Which creates training patterns like:

In [15]:
## Listing 27.15

Each element in the sequence is then provided as a new input feature to the network. This requires a modification of how the input sequences reshaped in the data preparation step:

In [16]:
## Listing 27.16

It also requires a modification for how the sample patterns are reshaped when demonstrating predictions from the model.

In [17]:
## Listing 27.17

The entire code listing is provided below for completeness.

In [18]:
## Listing 27.18

Running this example provides the following output.

In [19]:
## Listing 27.19

We can see a small lift in performance that may or may not be real. This is a simple problem that we could not learn with LSTMs, even with the window method. Again, this is a misuse of the LSTM network by a poor framing of the problem. Indeed, the sequences of letters are time steps of one feature rather than one time step of separate features. We have given more context to the network but not more sequence as expected. In the next section, we will give more context to the network in time steps.

## LSTM for a Time Step Window to One-Char Mapping

In Keras, the intended use of LSTMs is to provide the context in the form of time steps rather than windowed features like with other network types. We can take our first example and change the sequence length from 1 to 3.

In [20]:
## Listing 27.20

Again, this creates input-output pairs that look like:

In [21]:
## Listing 27.21

The difference is that the reshaping of the input data takes the sequence as a time step sequence of one feature rather than a single time step of multiple features.

In [22]:
## Listing 27.22

This is the intended use of providing sequence context to your LSTM in Keras. The complete code example is provided below for completeness.

In [23]:
## Listing 27.23

Running this example provides the following output.

In [24]:
## Listing 27.24

We can see that the model learns the problem perfectly, as evidenced by the model evaluation and the example predictions. But it has learned a simpler problem. Specifically, it has learned to predict the next letter from a sequence of three letters in the alphabet. It can be shown any random sequence of three letters from the alphabet and predict the next letter.

It cannot enumerate the alphabet. I expect that a larger enough Multilayer Perceptron network might learn the same mapping using the window method. The LSTM networks are stateful. They should learn the whole alphabet sequence, but by default, the Keras implementation resets the network state after each training batch.

## LSTM State Maintained Between Samples Within A Batch

The Keras implementation of LSTMs resets the state of the network after each batch. This suggests that if we had a large batch size to hold all input patterns and if all the input patterns were ordered sequentially, the LSTM could use the sequence's context to learn the sequence better. We can demonstrate this easily by modifying the first example for learning a one-to-one mapping and increasing the batch size from 1 to the size of the training dataset. Additionally, Keras shuffles the training dataset before each training epoch. To ensure the training data patterns remain sequential, we can disable this shuffling.

In [25]:
## Listing 27.25

The network will learn the mapping of characters using the within-batch sequence, but this context will not be available to the network when making predictions. We can evaluate both the ability of the network to make predictions randomly and in sequence. The full code example
is provided below for completeness.

In [26]:
## Listing 27.26

Running the example provides the following output.

In [27]:
## Listing 27.27

As we expected, the network can use the within-sequence context to learn the alphabet, achieving 100% accuracy on the training data. Notably, the network can make accurate predictions for the next letter in the alphabet for randomly selected characters. Very impressive.

## Stateful LSTM for a One-Char to One-Char Mapping

We have seen that we can break up our raw data into fixed-size sequences and that the LSTM can learn this representation, but only to learn random mappings of 3 characters to 1 character. We have also seen that we can pervert batch size to offer more sequence to the network, but only during training. Ideally, we want to expose the network to the entire sequence and let it learn the inter-dependencies, rather than us define those dependencies explicitly in
the framing of the problem.

We can do this in Keras by making the LSTM layers stateful and manually resetting the state of the network at the end of the epoch, which is also the end of the training sequence. This is truly how the LSTM networks are intended to be used. We first need to define our LSTM layer as stateful. In so doing, we must explicitly specify the batch size as a dimension on the input shape. This also means that we must also specify and adhere to this same batch size when we evaluate the network or make predictions. This is not a problem now as we are using a batch size of 1. This could introduce difficulties when making predictions when the batch size is not one as predictions will need to be made in batch and in sequence.

In [28]:
## Listing 27.28

An important difference in training the stateful LSTM is that we train it manually one epoch at a time and reset the state after each epoch. We can do this in a for loop. Again, we do not shuffle the input, preserving the sequence in which the input training data was created.

In [29]:
## Listing 27.29

As mentioned, we specify the batch size when evaluating the performance of the network on the entire training dataset.

In [30]:
## Listing 27.30

Finally, we can demonstrate that the network has indeed learned the entire alphabet. We can seed it with the first letter A, request a prediction, feed the prediction back in as an input, and repeat the process all the way to Z.

In [31]:
## Listing 27.31

We can also see if the network can make predictions starting from an arbitrary letter.

In [32]:
## Listing 27.32

The entire code listing is provided below for completeness.

In [33]:
## Listing 27.33

Running the example provides the following output.

In [34]:
## Listing 27.34

We can see that the network has memorized the entire alphabet perfectly. It used the samples' context and learned whatever dependency it needed to predict the next character in the sequence. We can also see that if we seed the network with the first letter, it can correctly rattle off the rest of the alphabet. We can also see that it has only learned the full alphabet sequence and that from a cold start. When asked to predict the next letter from K that it predicts B and falls back into regurgitating the entire alphabet. To truly predict
K, the network state would need to be warmed up iteratively fed the letters from A to J. This tells us that we could achieve the same effect with a stateless LSTM by preparing training data like:

In [35]:
## Listing 27.35

The input sequence is fixed at 25 (a-to-y to predict z), and patterns are prefixed with zero paddings. Finally, this raises the question of training an LSTM network using variable-length input sequences to predict the next character.

## LSTM with Variable Length Input to One-Char Output

In the previous section, we discovered that the Keras stateful LSTM was only a shortcut to replaying the first n-sequences but didn't help us learn a generic alphabet model. This section explores a variation of the stateless LSTM that learns random subsequences of the alphabet and an effort to build a model that can be given arbitrary letters or subsequences of letters and predict the next letter in the alphabet.

Firstly, we are changing the framing of the problem. To simplify, we will define a maximum input sequence length and set it to a small value like 5 to speed up training. This defines the maximum length of subsequences of the alphabet which will be drawn for training. In extensions, this could just be set to the entire alphabet (26) or longer if we allow looping back to the start of the sequence. We also need to define the number of random sequences to create, in this case, 1,000. This, too, could be more or less. I expect fewer patterns are required.

In [36]:
## Listing 27.36

Running this code in the broader context will create input patterns that look like the following:

In [37]:
## Listing 27.37

The input sequences vary in length between 1 and max_len and therefore require zero paddings. Here, we use the left-hand-side (prefix) padding with the Keras built-in `pad_sequences()` function.

In [38]:
## Listing 27.38

The trained model is evaluated on randomly selected input patterns. This could just as easily be new randomly generated sequences of characters. I also believe this could also be a linear sequence seeded with A with outputs fed back in as single character inputs. The complete code listing is provided below for completeness.

In [39]:
## Listing 27.39

Running this code produces the following output:

In [40]:
## Listing 27.40

We can see that although the model did not learn the alphabet perfectly from the randomly generated subsequences, it did very well. The model was not tuned and may require more training, a larger network, or both (an exercise for the reader). This is a good natural extension to the all sequential input examples in each batch alphabet model learned above in that it can handle ad hoc queries, but this time of arbitrary sequence length (up to the max length).

## Summary

In this lesson, you discovered LSTM recurrent neural networks in Keras and how they manage state. Specifically, you learned:
* How to develop a naive LSTM network for one-character to one-character prediction.
* How to configure a naive LSTM to learn a sequence across time steps within a sample.
* How to configure an LSTM to learn a sequence across samples by manually managing state.